# Humanoid AI Gym

This notebook is a template for training the humanoid robot.


In [1]:
import numpy as np
from gym.envs.mujoco import mujoco_env
from gym import utils

DependencyNotInstalled: No module named 'mujoco_py'. (HINT: you need to install mujoco_py, and also perform the setup instructions here: https://github.com/openai/mujoco-py/.)

The following variables are used for defining the actions and states of the game "LunarLander-v2"

In [3]:
STATE_SPACE = 8
ACTION_SPACE = 4
ENV = gym.make("LunarLander-v2")

# actions
DO_NOTHING = 0
LEFT_ENGINE = 1
MAIN_ENGINE = 2
RIGHT_ENGINE = 3

# state
X_POS = 0
Y_POS = 1
X_SPEED = 2
Y_SPEED = 3
ANGLE = 4
ANGLE_SPEED = 5
FIRST_LEG = 6
SECOND_LEG = 7


Now we'll define the basic interface with the game. The game is essentially a very rapid turn-based game. In each round, there are 2 main steps:
1. An action is taken by the agent.
2. The game updates according to its current state and the input action from the agent.

In [4]:
def mass_center(model, sim):
    mass = np.expand_dims(model.body_mass, 1)
    xpos = sim.data.xipos
    return (np.sum(mass * xpos, 0) / np.sum(mass))[0]

class HumanoidEnv(mujoco_env.MujocoEnv, utils.EzPickle):
    def __init__(self):
        mujoco_env.MujocoEnv.__init__(self, 'humanoid.xml', 5)
        utils.EzPickle.__init__(self)

    def _get_obs(self):
        data = self.sim.data
        return np.concatenate([data.qpos.flat[2:],
                               data.qvel.flat,
                               data.cinert.flat,
                               data.cvel.flat,
                               data.qfrc_actuator.flat,
                               data.cfrc_ext.flat])

    def step(self, a):
        pos_before = mass_center(self.model, self.sim)
        self.do_simulation(a, self.frame_skip)
        pos_after = mass_center(self.model, self.sim)
        alive_bonus = 5.0
        data = self.sim.data
        lin_vel_cost = 1.25 * (pos_after - pos_before) / self.dt
        quad_ctrl_cost = 0.1 * np.square(data.ctrl).sum()
        quad_impact_cost = .5e-6 * np.square(data.cfrc_ext).sum()
        quad_impact_cost = min(quad_impact_cost, 10)
        reward = lin_vel_cost - quad_ctrl_cost - quad_impact_cost + alive_bonus
        qpos = self.sim.data.qpos
        done = bool((qpos[2] < 1.0) or (qpos[2] > 2.0))
        return self._get_obs(), reward, done, dict(reward_linvel=lin_vel_cost, reward_quadctrl=-quad_ctrl_cost, reward_alive=alive_bonus, reward_impact=-quad_impact_cost)

    def reset_model(self):
        c = 0.01
        self.set_state(
            self.init_qpos + self.np_random.uniform(low=-c, high=c, size=self.model.nq),
            self.init_qvel + self.np_random.uniform(low=-c, high=c, size=self.model.nv,)
        )
        return self._get_obs()

    def viewer_setup(self):
        self.viewer.cam.trackbodyid = 1
        self.viewer.cam.distance = self.model.stat.extent * 1.0
        self.viewer.cam.lookat[2] = 2.0
        self.viewer.cam.elevation = -20

Finally, we'll define a couple of test agents to play with.

In [5]:
def random_agent(state):
    """This agent returns random actions."""
    return int(np.random.rand() * 4)


def stupid_agent(state):
    """A very simple expert system."""
    if state[FIRST_LEG] == 1 and state[SECOND_LEG] == 1:
        return DO_NOTHING
    if state[Y_SPEED] < -0.2:
        return MAIN_ENGINE
    if state[ANGLE] < -0.1:
        return LEFT_ENGINE
    if state[ANGLE] > 0.1:
        return RIGHT_ENGINE
    return DO_NOTHING

In [13]:
my_agent = stupid_agent
play_game(my_agent, ENV, show=True, verbose=False)

216.4407724587458

In [ ]:
def train(n, agent):
    cum_fitness = 0
    for i in range(n):
        fitness = play_game(agent, ENV, show=False, verbose=False)
        cum_fitness += fitness
    return cum_fitness
